In [0]:
###################################################
# ラボワーク　ベースライン訓練スクリプト
###################################################

In [80]:
# colab用のスクリプト
from google.colab import drive 
drive.mount('/content/drive')
%cd drive/My\ Drive/labowork

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/labowork'
/content/drive/My Drive/labowork


In [0]:
# import
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import make_scorer
from sklearn.base import clone
import pickle

In [0]:
# 前処理済みデータの読みこみ
X = np.load("./temp/preprocessed_train.npy")
y = np.load("./temp/target.npy")

In [0]:
# 評価関数の実装
def R2(y_true, y_predict):
  y_mean = np.mean(y_true)
  return 1 - ((y_true - y_predict) ** 2).sum() / ((y_true - y_mean) ** 2).sum()
score = make_scorer(R2, greater_is_better=True)

In [0]:
# train / testにsplit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [0]:
# GridSearch用定数の宣言
cv_num = 5
hyper_parameters = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5]
}
seed = 64
model_base = RandomForestRegressor(n_jobs=-1, random_state=seed)

In [0]:
# train -> train（訓練） / validation（hyper parametersの評価）
# cross validation
# train: 0, 1, 2, 3 -> train 0, 1, 2 val: 3 RandomForest<- 並列可能
# train 0, 1, 3 val: 2
# train 0, 2, 3 val 1
# 

In [62]:
# GridSearchの実行
model = GridSearchCV(model_base, hyper_parameters, scoring=score, n_jobs=1, cv=KFold(cv_num, True, seed), verbose=10)
model.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] max_depth=3, n_estimators=50 ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........ max_depth=3, n_estimators=50, score=0.290, total= 2.1min
[CV] max_depth=3, n_estimators=50 ....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV] ........ max_depth=3, n_estimators=50, score=0.278, total= 2.1min
[CV] max_depth=5, n_estimators=50 ....................................


...

[CV] ........ max_depth=5, n_estimators=50, score=0.366, total= 3.3min
[CV] max_depth=5, n_estimators=50 ....................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  7.5min remaining:    0.0s


[CV] ........ max_depth=5, n_estimators=50, score=0.356, total= 3.3min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 10.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 10.8min finished


GridSearchCV(cv=KFold(n_splits=2, random_state=64, shuffle=True),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=-1,
                                             oob_score=False, random_state=64,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=1,
   

In [67]:
print(model.best_params_)
print(model.best_score_)

{'max_depth': 5, 'n_estimators': 50}
0.36113117504621006


In [69]:
# test dataへのscoreを導出
y_predict = model.predict(X_test)
print(R2(y_test, y_predict))

0.37203319491344744


In [77]:
# testデータを入れて学習しなおす
model_fin = clone(model_base)
model_fin.set_params(**model.best_params_)
model_fin.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                      oob_score=False, random_state=64, verbose=0,
                      warm_start=False)

In [0]:
# 作成モデルの保存
with open("./model/basemodel.pkl", "wb") as f:
  pickle.dump(model_fin, f)